In [ ]:
import pandas as pd
import numpy as np
import xlrd
import math
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import os
import json

In [ ]:
# Read data
path_json = "./data/validated/"
json_files = [pos_json for pos_json in os.listdir(path_json) if pos_json.endswith('.json')]
        
text_list = []
index = 0
null_count = 0
for file_name in json_files:
    path = path_json + file_name
    with open(path, 'r') as f:
        data = json.load(f)
        if('summary' in data):
            summary = (data.get('summary'))
            text_list.append(summary)
        else:
            null_count += 1
            print(index)
            print(json_files[index])
    index += 1
print(null_count) #Prints out number of entries with no summary


In [ ]:
# This cell is just for random checking of data
'''
with open(path_json+json_files[16], 'r') as file:
    data = json.load(file)
    print('action' in data)
'''
print(len(text_list))
print(text_list[14:18])
#json_files[14:18]

In [ ]:
#Final all unique words in a list of texts
def allUniqueWords(docList):
    uniq = []
    for text in docList:
        terms = text.split()
        for word in terms:
            word = word.lower()
            if word not in uniq:
                uniq.append(word)
    return uniq

In [ ]:
#Compute Term Frequency
def computeTF(docList, unique):
    data = pd.DataFrame(columns=unique)
    for i in range(len(docList)):
        data.loc[i] = np.zeros(len(unique))
    row = 0
    col = 0
    for text in docList:
        terms = text.split()
        for word in terms:
            word = word.lower()
            data.loc[row, word] += 1 
        row += 1
    return data

In [ ]:
#Compute TFIDF (Term Frequency - Inverse Document Frequency)
def computeTFIDF(TF, unique):
    doc_data = pd.DataFrame(columns=unique)
    doc_data.loc[0] = np.zeros(len(unique))
    #print(doc_data)
    N = TF.shape[0]
    for i in range(N):
        for word in TF.columns:
            print(word)
            if (TF.loc[i, word] > 0):
                doc_data.loc[0, word] += 1
    for word in doc_data.columns:
        doc_data.loc[0, word] = math.log10(N / (1 + doc_data.loc[0, word]))
    for word in TF.columns:
        for i in range(N):
            TF.loc[i, word] = TF.loc[i, word] * doc_data.loc[0,word]
    return TF

In [ ]:
#Finds the indexes of the N greatest element of a list
def Nmaxelements(list1, N): 
    final_list = [] 
  
    for i in range(0, N):  
        max1 = 0
        maxi = -1
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j] 
                maxi = j 
        list1.remove(max1); 
        final_list.append(maxi) 
    return(final_list)

In [ ]:
#Finds the pronouns in a block of text
def allPronoun(text):
    pro = []
    terms = text.split()
    for word in terms:
        if(not word.islower):
            print(word)
            pro.append(word)
    return pro

In [ ]:
#Finds all unique words and term frequency (Warning: Takes forever to run)
vcdb_unique = allUniqueWords(text_list)
vcdb_TF = computeTF(text_list, vcdb_unique)

In [ ]:
#Create short version of data by random index
import random
def shortVersionRand(short_size, docList):
    short_list = []
    rand_nums = random.sample(range(len(docList)), short_size)
    for i in range(short_size):
        short_list.append(docList[i])
    return short_list

In [ ]:
#Create short version of data by choosing those with equal or more than N sentences
def shortVersionLong(docList, N):
    short_list = []
    for text in docList:
        sentences = text.split(".")
        if(len(sentences) >= N):
            short_list.append(text)
    return short_list

In [ ]:
shortRand = shortVersionRand(100, text_list)
shortLong = shortVersionLong(text_list, 3)
print(len(shortRand))
print(shortRand[:5])
print(len(shortLong))
print(shortLong[:5])

In [ ]:
#Finds all unique words and term frequency on short list of random index
vcdb_unique_rand = allUniqueWords(shortRand)
vcdb_TF_rand = computeTF(shortRand, vcdb_unique_rand)

In [ ]:
vcdb_unique_rand

In [ ]:
vcdb_TF_rand

In [ ]:
TFIDF = computeTFIDF(vcdb_TF_rand, vcdb_unique_rand) 

In [ ]:
TFIDF

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 

In [ ]:
#Utilizes tokenization (word2vec) and generates CBOW (Continuous Bag of Words) and Skip Gram model
#CURRENTLY NOT WORKING AS INTENDED
data = [] 
f = " ".join(story for story in text_list)
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

# Create CBOW model

model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5) 
# Create skip gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, 
                                             window = 5, sg = 1) 

In [ ]:
def genWordCloud(docList):
    all_text = " ".join(story for story in docList)
    wordcloud = WordCloud().generate(all_text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
genWordCloud(text_list)

<b>Below is Analysis Done for the Old Dataset</b>
<br>


In [ ]:
pr = pd.read_excel("./data/Privacy Rights Clearinghouse Top Breaches.xlsx")
br = pd.read_excel("./data/Balloon Race Data Breaches Filtered.xlsx")
#Names columns correctly (Run only once)
pr.columns = pr.loc[0,:]
pr = pr.loc[1:,:]

br.columns = br.loc[0,:]
br = br.loc[2:,:]

In [ ]:
br_stories = br.loc[:,'story']
pr_stories = pr.loc[:,'Description of incident']
stories = []
stories = br_stories.tolist() + pr_stories.tolist()
len(stories)

In [ ]:
# Take out uninteresting story (columns with interesting story not labeled 'y')
boring = pr.loc[413, 'Description of incident'][:33]
boring
pr_interesting = []
for story in pr_stories:
    if(story[:33] != boring):
        pr_interesting.append(story)
len(pr_interesting)

br_interesting = br.loc[br['interesting story'] == 'y'].loc[:,'story']
interesting_stories = pr_interesting + br_interesting.tolist()
len(interesting_stories)

In [ ]:
unique = allUniqueWords(stories)
TF = computeTF(stories, unique)
TF

In [ ]:
TF.shape

In [ ]:
TFIDF_story = computeTFIDF(TF, unique)
TFIDF

In [ ]:
#Runs TFIDF (Warning: Takes forever)
unique = allUniqueWords(interesting_stories)
TFIDF_interesting = computeTFIDF(computeTF(interesting_stories, unique), unique)

In [ ]:
genWordCloud(stories)

In [ ]:
genWordCloud(interesting_stories)